In [ ]:
import pandas as pd
from pingouin import intraclass_corr
from tqdm.auto import tqdm

df = pd.read_excel("interreader_features.xlsx")

start_feature = "original_shape_Elongation"
end_feature = "lbp-3D-k_ngtdm_Strength"

columns = df.columns.tolist()
start_idx = columns.index(start_feature)
end_idx = columns.index(end_feature)
feature_cols = columns[start_idx:end_idx + 1]

icc_results = []

for feature in tqdm(feature_cols, desc="Calculating ICC"):
    temp_df = df[['Scan_Name', 'Segment Name', 'Anon_Name', feature]].copy()
    temp_df.rename(columns={
        'Anon_Name': 'rater',
        'Scan_Name': 'scan',
        'Segment Name': 'segment',
        feature: 'score'
    }, inplace=True)

    # Compute ICC(2,1)
    icc = intraclass_corr(data=temp_df, targets='segment', raters='rater', ratings='score')
    icc_row = icc.loc[icc['Type'] == 'ICC2']

    icc_results.append({
        'Feature': feature,
        'ICC': icc_row['ICC'].values[0],
        'CI95_Lower': icc_row['CI95%'].values[0][0],
        'CI95_Upper': icc_row['CI95%'].values[0][1],
        'F': icc_row['F'].values[0],
        'df1': icc_row['df1'].values[0],
        'df2': icc_row['df2'].values[0],
        'pval': icc_row['pval'].values[0]
    })

icc_df = pd.DataFrame(icc_results).sort_values(by='ICC', ascending=False).reset_index(drop=True)

icc_df.to_excel("ICC_results.xlsx", index=False)
icc_df.to_csv("ICC_results.csv", index=False)

icc_df.head()

                          Feature  ICC  CI95_Lower  CI95_Upper             F  \
0  wavelet-LLH_firstorder_Maximum  1.0         1.0         1.0 -2.463079e+15   
1  wavelet-LHL_firstorder_Maximum  1.0         1.0         1.0 -2.520410e+15   
2    logarithm_firstorder_Maximum  1.0         1.0         1.0 -8.187500e+15   
3      lbp-3D-m2_firstorder_Range  1.0         1.0         1.0 -8.446521e+15   
4   squareroot_firstorder_Maximum  1.0         1.0         1.0 -3.874209e+15   

   df1  df2  pval  
0    3    3   1.0  
1    3    3   1.0  
2    3    3   1.0  
3    3    3   1.0  
4    3    3   1.0  
